In [1]:
import numpy as np
import matplotlib
import mne
import pandas as pd

In [2]:
import os
import pywt
import numpy as np
import pandas as pd
from mne.io import read_raw_fif
from mne import create_info
from mne.io import RawArray
from mne.epochs import Epochs
from mne.time_frequency import tfr_morlet
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import re
import os
from mne import create_info, concatenate_raws, make_fixed_length_epochs

In [3]:
# for band_folder in ['delta_band_model', 'theta_band_model', 'alpha_band_model', 'beta_band_model', 'gamma_band_model']:
#     if not os.path.exists(band_folder):
#         folder =os.path.join("R:\dens\VA\svm_model_mobilenet",band_folder)
#         os.makedirs(folder)

In [1]:
# # # # 22,69,66,46,107
# for band_folder in ['21','22','32','23','10','27','12','44','35','46','36','57','51','66','69','8','2','121','123','116','111','107','103','97','86','95','91','61','76','82','74']:
# # for band_folder in ['22','69','66','46','107']:
#     if not os.path.exists(band_folder):
#         folder =os.path.join("R:\dens\VA\svm_model_resnet\\rhythm_band_model",band_folder)
#         os.makedirs(folder)

In [5]:
# Define function to determine label based on valence
def determine_label(valence):
    return 'HV' if valence > 5 else 'LV'

In [6]:
# Function to perform wavelet decomposition and extract EEG rhythms
def extract_eeg_rhythms(eeg_data, channel_name,band_name):
    # Define wavelet parameters
    wavelet = 'db4'  # Daubechies 4th order wavelet
    levels = 6

    # Perform wavelet decomposition
    coeffs = pywt.wavedec(eeg_data, wavelet, level=levels)
    
    rhythm = coeffs[levels]
    
    return rhythm

In [7]:
def apply_continuous_wavelet_transform_to_raw(raw_data, frequencies, sfreq):
    # Create info object for a single-channel EEG signal
    info = mne.create_info(ch_names=['channel1'], sfreq=sfreq, ch_types='eeg')
#     print(info)
    # Reshape raw_data to a 2D array
    raw_data_2d = raw_data.reshape(1, -1)

    # Create RawArray object from the raw data and info
    raw_array = mne.io.RawArray(raw_data_2d, info)
#     print(raw_array.get_data())
    # Create an Epochs object from the RawArray
    events = np.array([[0, 0, 1]])  # Dummy event
    epochs = mne.EpochsArray([raw_data_2d], info, events=events, tmin=0)

    # Apply continuous wavelet transform to epochs
    tfr = mne.time_frequency.tfr_morlet(epochs, freqs=frequencies, n_cycles=frequencies / 2.0, return_itc=False, average=False)
    print(tfr.data.shape)
    return tfr

In [8]:
channels_list = [f'{i+1}' for i in range(128)]

In [9]:
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt

def convert_tfr_to_images(tfr, labels, channel_names):

    if tfr.data.shape[0] > 0:
        power_data = tfr.data[0, :, :, :]
        power_data_normalized = 10 * np.log10(power_data)
        
        # Normalize the image data
        normalized_image = (power_data_normalized - np.min(power_data_normalized)) / np.ptp(power_data_normalized) * 255.0
        normalized_image_uint8 = normalized_image.astype(np.uint8)
#         print(normalized_image_uint8[0])
        # Resize the image to 224x224 RGB format
        resized_images = []
        for img in normalized_image_uint8:
            img_reshaped = np.repeat(img[:, :, np.newaxis], 3, axis=2)  # Convert to 224x224x3 RGB format
            img_pil = Image.fromarray(img_reshaped)
            resized_images.append(img_pil.resize((224, 224)))
            
        return resized_images
    else :
        return False 
    


In [10]:
# Define function to load and preprocess an image
def load_and_preprocess_image(img_path):

    img_array = tf.keras.preprocessing.image.img_to_array(img_path)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
#     print(img_array)
    return img_array

# Define function to extract deep features using a pre-trained model
def extract_deep_features(model, img_path):
    img_array = load_and_preprocess_image(img_path)
    features = model.predict(img_array)
    return features.flatten()

In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.optimizers import Adam
import scipy
# Define channels
channels = [21,22,32,23,10,27,12,44,35,46,36,57,51,66,69,8,2,121,123,116,111,107,103,97,86,95,91,61,76,82,74]
frequencies = np.arange(1, 41, 1)
# Load CSV file
csv_path = 'R:/EEG_Data/cleaned_data_readings.csv'
data = pd.read_csv(csv_path)


# Initialize empty DataFrame to store labels
labels_df = pd.DataFrame(columns=['file_name', 'channel', 'label'])


image =[]
lab = []

# Loop through each row in the CSV file
for index, row in tqdm(data.iterrows(), total=len(data), desc='Processing EEG files'):
    file_name = row['file_name']
    valence = row['rating_valence']
#     arousal = row['arousal']
#     group = row['Groups']
    
    labels = group

    # Form the EEG file path
    eeg_file_path = os.path.join('R:\\EEG_Data\\data', f"{file_name}")

    # Load EEG data (replace this with your actual EEG data loading code)
    raw = scipy.io.loadmat(eeg_file_path)
#     print(raw['data'])
    eeg_data = []
    for i in channels:

        eeg_data.append(raw['data'][i])

    # For each channel, extract rhythms for each band
    for idx , channel in enumerate(channels):

        lab.append(labels)
        # Convert rhythms to time-frequency representation  (rhythms, frequencies, sfreq, file, channel_names)
        tfr = apply_continuous_wavelet_transform_to_raw(eeg_data[idx],frequencies, 250)
#         print(tfr.data)
#         Convert TFR to images  (tfrs, save_path, labels, channel_names)
        images = convert_tfr_to_images(tfr,labels,channel)
        image.append(images)



Processing EEG files:   0%|          | 0/418 [00:00<?, ?it/s]

Creating RawArray with float64 data, n_channels=1, n_times=876
    Range : 0 ... 875 =      0.000 ...     3.500 secs
Ready.
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1 events and 876 original time points ...
0 bad epochs dropped
Not setting metadata


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


1/1 [==============================] - 1s 833ms/step
Creating RawArray with float64 data, n_channels=1, n_times=876
    Range : 0 ... 875 =      0.000 ...     3.500 secs
Ready.
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1 events and 876 original time points ...
0 bad epochs dropped
Not setting metadata


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


1/1 [==============================] - 1s 963ms/step
Creating RawArray with float64 data, n_channels=1, n_times=876
    Range : 0 ... 875 =      0.000 ...     3.500 secs
Ready.
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1 events and 876 original time points ...
0 bad epochs dropped
Not setting metadata


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


1/1 [==============================] - 1s 929ms/step


Processing EEG files: 100%|██████████| 418/418 [1:31:49<00:00, 13.18s/it]


In [ ]:
len(lab)

In [ ]:
image[0][0]

In [18]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Reshape
from sklearn.preprocessing import LabelEncoder

# Constants
number_of_classes = 150528  
batch_size = 32  
image_size = (224, 224, 3)

# Load MobileNetV2 as the base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
images_array = np.array([np.array(img[0]) for img in image])
# Freeze the base model
base_model.trainable = False



base_model_input = GlobalAveragePooling2D()(base_model.output)
custom_input = Dense(512, activation='relu')(base_model_input)


custom_output = Dense(150528, activation='sigmoid')(custom_input)
model_output = Reshape((224, 224, 3))(custom_output)

# Construct the full model
model = Model(inputs=base_model.input, outputs=model_output)

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
history = model.fit(images_array, images_array, epochs=10, batch_size=batch_size)


Processing EEG files:   0%|          | 0/418 [00:00<?, ?it/s]

Epoch 1/50
1/1 [==============================] - 0s 446ms/step - loss: 0.7806
Epoch 2/50
1/1 [==============================] - 0s 6ms/step - loss: 0.1373
Epoch 3/50
1/1 [==============================] - 0s 7ms/step - loss: -0.6303
Epoch 4/50
1/1 [==============================] - 0s 6ms/step - loss: -1.7995
Epoch 5/50
1/1 [==============================] - 0s 6ms/step - loss: -3.4219
Epoch 6/50
1/1 [==============================] - 0s 6ms/step - loss: -5.5196
Epoch 7/50
1/1 [==============================] - 0s 6ms/step - loss: -8.1131
Epoch 8/50
1/1 [==============================] - 0s 6ms/step - loss: -11.2207
Epoch 9/50
1/1 [==============================] - 0s 5ms/step - loss: -14.8598
Epoch 10/50
1/1 [==============================] - 0s 7ms/step - loss: -19.0480
Epoch 11/50
1/1 [==============================] - 0s 6ms/step - loss: -23.8038
Epoch 12/50
1/1 [==============================] - 0s 6ms/step - loss: -29.1456
Epoch 13/50
1/1 [==============================] - 0s 6m

Processing EEG files:   0%|          | 0/418 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'R:\\dens\\VA\\svm_model_resnet\\encoded_data\\22\\Encoded_Amused_mit004Trial-3Click-1-slor.mat.npy'

In [ ]:
len(eeg_data[0])

In [ ]:
len(encoded_data[0])

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Reshape
from sklearn.preprocessing import LabelEncoder

# Constants
number_of_classes = 150528  # Number of pixels in 224x224x3 image
batch_size = 32  # Adjust this according to your system's capability
image_size = (224, 224, 3)

# Load MobileNetV2 as the base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
images_array = np.array([np.array(img[0]) for img in image])
# Freeze the base model
base_model.trainable = False


base_model_input = GlobalAveragePooling2D()(base_model.output)
custom_input = Dense(512, activation='relu')(base_model_input)


custom_output = Dense(150528, activation='sigmoid')(custom_input)
base_model_output = Reshape((224, 224, 3))(custom_output)

# Construct the full model
model = Model(inputs=base_model.input, outputs=base_model_output)

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
history = model.fit(images_array, images_array, epochs=10, batch_size=batch_size)


In [ ]:
model.save('R:\\dens\\VA\\svm_model_resnet\\rhythm_band_model\\my_model.h5')

In [ ]:
import os
import numpy as np
import pandas as pd
from mne import EpochsArray, create_info
from scipy import signal
from scipy.io import savemat
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tqdm import tqdm
import scipy


rhythm_frequencies = np.arange(1, 41, 1)


# Define channels
channels = [21,22,32,23,10,27,12,44,35,46,36,57,51,66,69,8,2,121,123,116,111,107,103,97,86,95,91,61,76,82,74]

# Load CSV file
csv_path = 'R:/EEG_Data/cleaned_data_readings_valence.csv'
data = pd.read_csv(csv_path)


# Initialize empty DataFrame to store labels
labels_df = pd.DataFrame(columns=['file_name', 'channel', 'label'])


# image =[]


# Loop through each row in the CSV file
for index, row in tqdm(data.iterrows(), total=len(data), desc='Processing EEG files'):
    file_name = row['file_name']
    valence = row['rating_valence']
    arousal = row['arousal']
    
    labels = determine_label(valence)

    # Form the EEG file path
    eeg_file_path = os.path.join('R:\\EEG_Data\\data', f"{file_name}")

    # Load EEG data (replace this with your actual EEG data loading code)
    raw = scipy.io.loadmat(eeg_file_path)
#     print(raw['data'])
    eeg_data = []
    for i in channels:

        eeg_data.append(raw['data'][i])

    # For each channel, extract rhythms for each band
    for idx , channel in enumerate(channels):
        for band_name, frequencies in zip(['rhythm'],
                                          [ rhythm_frequencies
                                          ]):

            # Extract rhythms for each band (replace this with your actual rhythm extraction code)
            rhythms = extract_eeg_rhythms(eeg_data[idx], frequencies,band_name)
#             print(rhythms)
            # Label rhythms based on valence
            rhythm_labels = labels
            
            # Convert rhythms to time-frequency representation  (rhythms, frequencies, sfreq, file, channel_names)
            tfr = apply_continuous_wavelet_transform_to_raw(eeg_data[idx],frequencies, 250)
#             print(tfr.data)
#             Convert TFR to images  (tfrs, save_path, labels, channel_names)
            images = convert_tfr_to_images(tfr,labels,channel)
#             image.append(images)
# # #             print(images[0].shape)

#             if images:
            
#             mobilenet_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#             for layer in mobilenet_model.layers:
#                 layer.trainable = False
            for layer in model.layers:
                layer.trainable = False

            # Extract deep features using VGG16
            deep_features = extract_deep_features(model, images[0])
           
            features_directory = f'R:\dens\VA\svm_model_resnet\\{band_name}_band_model\\{channel}'
            # Save the features as a NumPy array for the current band
            features_path = os.path.join(features_directory, f"ResNet50_{file_name}_label_{labels}.npy")
            np.save(features_path, deep_features)

            


In [ ]:
print(deep_features)
print(len(deep_features))

In [ ]:
print(model.summary())

In [ ]:

plt.imshow(image[0][0])
plt.axis('off')
plt.show()

In [ ]:
import os
import numpy as np
from sklearn.decomposition import PCA
import shutil
from sklearn.preprocessing import StandardScaler

In [ ]:
channels = [21,22,32,23,10,27,12,44,35,46,36,57,51,66,69,8,2,121,123,116,111,107,103,97,86,95,91,61,76,82,74]
folder_path = r"R:\dens\VA\svm_model_resnet\rhythm_band_model"
for channel in channels:
    file_path=os.path.join(folder_path,str(channel))
#     print(file_path)
    variance_thresholds = [0.70, 0.75, 0.80, 0.85, 0.90, 0.95]
    for threshold in variance_thresholds:
        threshold_folder = os.path.join(file_path, f"pca_{threshold}")
        os.makedirs(threshold_folder, exist_ok=True)

    
    


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

channels = [21,22,32,23,10,27,12,44,35,46,36,57,51,66,69,8,2,121,123,116,111,107,103,97,86,95,91,61,76,82,74]
fo_path = r"R:\dens\VA\svm_model_resnet\rhythm_band_model"
for channel in channels:
    folder_path=os.path.join(fo_path,str(channel))
    

# Define PCA values
    pca_values = [0.70, 0.75, 0.80, 0.85, 0.90, 0.95]

# Load data from .npy files into a dataframe
    data_dict = {}
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_path.endswith(".npy"):
            data = np.load(file_path)
            data_dict[file_name] = data.flatten()

    df = pd.DataFrame(data_dict)
    df = df.transpose()
# Apply PCA on the dataframe
    scaler = StandardScaler()
    data_standardized = scaler.fit_transform(df)

    pca_results = {}
    for pca_value in pca_values:
        pca = PCA(n_components=pca_value)
        reduced_data = pca.fit_transform(data_standardized)
        reduced_df = pd.DataFrame(reduced_data)
        save_folder = os.path.join(folder_path, f"pca_{pca_value}")
        save_path = os.path.join(save_folder, f"PCA{pca_value}.csv")
        reduced_features=scaler.fit_transform(reduced_df)
        reduced_df=pd.DataFrame(reduced_features)
        reduced_df.to_csv(save_path, index=False)
        for col_idx in reduced_df.columns:
            save_folder = os.path.join(folder_path, f"pca_{pca_value}")
            os.makedirs(save_folder, exist_ok=True)
            save_path = os.path.join(save_folder, f"{df.columns[col_idx]}.csv")
#         reduced_df[col_idx].to_csv(save_path, index=False)
        pca_results[pca_value] = pca.explained_variance_ratio_.sum()

    print("PCA results:", pca_results)


In [ ]:
reduced_df

In [ ]:
data = reduced_data
print(data.shape)
data

In [ ]:
data = np.load(r"R:\dens\VA\svm_model_mobilenet\rhythm_band_model\22\MobileNetV2_Adventorous_mit004Trial-4Click-3-slor.mat_label_HV.npy")
data.shape

In [ ]:
data